00 intro

In [1]:
import numpy as np
import plotly.graph_objects as go

from quantfreedom.enums import CandleBodyType
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.indicators.tv_indicators import rsi_tv

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="30m",
    candles_to_dl=300,
)

datetimes = candles[:, CandleBodyType.Timestamp].astype("datetime64[ms]")
close_prices = candles[:, CandleBodyType.Close]

In [3]:
rsi_is_below = 50
rsi_length = 14

In [4]:
rsi = rsi_tv(
    source=close_prices,
    length=rsi_length,
)
rsi = np.around(rsi,1)

In [5]:
prev_rsi = np.roll(rsi, 1)
prev_rsi[0] = np.nan

prev_prev_rsi = np.roll(prev_rsi, 1)
prev_prev_rsi[0] = np.nan

In [6]:
falling = prev_prev_rsi > prev_rsi
rising = rsi > prev_rsi
is_below = rsi < rsi_is_below

entries = np.where(is_below & falling & rising, True, False)
entry_signals = np.where(entries, rsi, np.nan)

exit_prices = np.full_like(rsi, np.nan)

In [7]:
fig = go.Figure()
# RSI
fig.add_scatter(
    x=datetimes,
    y=rsi,
    name="RSI",
    line_color="yellow",
)

# RSI Entries
fig.add_scatter(
    x=datetimes,
    y=entry_signals,
    mode="markers",
    name="Entries",
    marker=dict(
        size=12,
        symbol="circle",
        color="#00F6FF",
        line=dict(
            width=1,
            color="DarkSlateGrey",
        ),
    ),
)

# rsi below
fig.add_hline(
    y=rsi_is_below,
    opacity=0.3,
    line_color="red",
)

# update layout
fig.update_layout(
    height=500,
    xaxis_rangeslider_visible=False,
    title=dict(
        x=0.5,
        text="Long Signals",
        xanchor="center",
        font=dict(
            size=50,
        ),
    ),
)

# show
fig.show()